In [5]:
import requests
import pandas as pd
import numpy as np
import time
import os
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [7]:
clave_api= os.getenv("clave_api")

In [8]:
url = "https://api.themoviedb.org/3/discover/tv"
params ={"api_key": clave_api}
# Base URL para las imágenes
BASE_IMG_URL = "https://image.tmdb.org/t/p/w500"

In [5]:
url_generos = 'https://api.themoviedb.org/3/genre/tv/list'
try:
        # Hacer la solicitud a la API para obtener los géneros 
        genero_response = requests.get(url_generos, params=params) 
        genero_response.raise_for_status()  # Asegura que no haya error en la solicitud

        if genero_response.status_code == 200:
            generos = genero_response.json()['genres']
            # me quedo con el nombre del genero y el id para luego hacer match con la response en la que tengo solo el ID del genero
            generos_dict = {genero["id"]:genero['name'] for genero in generos} # // value de genero_id : value de genero_name
        else:
            print(f"Error {genero_response.status_code}: No se pudo obtener el genero")

except requests.exceptions.RequestException as e:
        print(f"Error al obtener géneros: {e}")

In [6]:
generos_dict

{10759: 'Action & Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 10762: 'Kids',
 9648: 'Mystery',
 10763: 'News',
 10764: 'Reality',
 10765: 'Sci-Fi & Fantasy',
 10766: 'Soap',
 10767: 'Talk',
 10768: 'War & Politics',
 37: 'Western'}

In [ ]:
# Función principal para obtener información de todas las series
def get_all_series(pages):
    todas_series = []
    processed_ids = set()  # Set para almacenar los IDs ya procesados
    url_generos = 'https://api.themoviedb.org/3/genre/tv/list'
    params ={"api_key": clave_api}
    
    try:
        # Hacer la solicitud a la API para obtener los GENEROS
        genero_response = requests.get(url_generos, params=params) 
        genero_response.raise_for_status()  # Asegura que no haya error en la solicitud

        if genero_response.status_code == 200:
            generos = genero_response.json()['genres']
            # me quedo con el nombre del genero y el id para luego hacer match con la response en la que tengo solo el ID del genero
            generos_dict = {genero["id"]:genero['name'] for genero in generos} # // value de genero_id : value de genero_name
        else:
            print(f"Error {genero_response.status_code}: No se pudo obtener el genero")

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener géneros: {e}")

    for page in range(1, pages + 1):
        url = "https://api.themoviedb.org/3/tv/popular"
        #url = "https://api.themoviedb.org/3/discover/tv"
        try:
            response = requests.get(url, params={**params, "page":page})
            response.raise_for_status()  # Asegura que no haya error en la solicitud
            # Verificamos cuántas peticiones quedan en el límite
            remaining_requests = int(response.headers.get("X-RateLimit-Remaining", 0))
            if remaining_requests == 0:
                print("Límite de peticiones alcanzado. Esperando...")
                time.sleep(10)  # Espera de 60 segundos si el límite es 0
        
            if response.status_code == 200:
                info_series = response.json()["results"]       
                
                for serie in info_series:
                    series_id = serie.get('id')  # ID de la serie
                    # Si ya procesamos esta serie, saltamos a la siguiente
                    if series_id in processed_ids:
                        continue
                    # Marcar el ID como procesado
                    processed_ids.add(series_id)

                    # Obtener información básica de la serie
                    title = serie.get("name")
                    synopsis = serie.get("overview")
                    rating = serie.get("vote_average")
                    first_air_date = serie.get("first_air_date")
                    last_air_date = serie.get("last_air_date")
                    genero_ids = serie.get("genre_ids") # me devuelve una lista de IDS de genero
                    popularity = serie.get("popularity")
                    original_language = serie.get("original_language")
                    origin_country = ", ".join([str(country) for country in serie.get("origin_country",[])if country]) if serie.get("origin_country") else "No data available"
                    number_ratings = serie.get("vote_count")
                    genero = ', '.join([generos_dict.get(genero_id, "Desconocido") for genero_id in genero_ids])
                    # Para cada genre_id, busco el nombre del género en el diccionario generos_dict
                    # La función get en los diccionarios me devuelve el valor asociado a una clave, 
                    # en este caso, el nombre del género correspondiente al genre_id.

                    # Quiero obtener la duración promedio de los episodios, el número de temporadas, el número de episodios y la plataforma
                    duracion_episodio = 0  # lo ponemos a 0
                    number_of_seasons = 0 # el número de temporadas también lo ponemos a 0
                    number_of_episodes = 0
                    production_companies = []
                    where_to_watch = []
                    reviews = []
                    presupuesto = 0
                    recaudacion = 0
                    cast = []
                    created_by = []
                    tagline =[]
                    in_production = []
                    status = []
                    vote_average = 0
                    poster_path = None
                    trailer = None

                    # solicitamos a la API detalles ADICIONALES (número de temporadas, número de episodios y duración)
                    url_series_id = f"https://api.themoviedb.org/3/tv/{series_id}"
                    details_response = requests.get(url_series_id, params=params)
                    if details_response.status_code == 200:
                            details = details_response.json()
                            # número de temporadas
                            number_of_seasons = details.get("number_of_seasons") 
                            # número de episodios
                            number_of_episodes = details.get("number_of_episodes")
                            # Duración de los episodios en minutos (si está disponible)
                            duracion_episodio = details.get("episode_run_time") 
                            if duracion_episodio:
                                duracion_episodio = sum(duracion_episodio) / len(duracion_episodio)  # Promedio de las duraciones
                            else:
                                duracion_episodio = 0  # Si no hay datos de duración ponemos 0  
                            # Extraer tagline, status y creadores
                            tagline = details.get("tagline", "No disponible")
                            in_production = details.get("in_production", "No disponible")
                            status = details.get("status", "No disponible")
                            created_by = [creator["name"] for creator in details.get("created_by")]
                            created_by = ", ".join(set(created_by))
                            vote_average = details.get("vote_average", "No disponible")
                            # Extraer las compañías productoras
                            production_companies = [company["name"] for company in details.get("production_companies")]
                            production_companies = ", ".join(set(production_companies))
                            # sacamos budget y recaudación
                            presupuesto = details.get("budget", "No data available")
                            recaudacion = details.get("revenue", "No data available")
                            poster_path = f"{BASE_IMG_URL}{details.get("poster_path")}"
    
                    
                    # solicitamos a la API la info de PLATAFORMAS donde ver las series
                    where_to_watch_url = f"https://api.themoviedb.org/3/tv/{series_id}/watch/providers"
                    donde_ver_response = requests.get(where_to_watch_url, params = params)
                    if donde_ver_response.status_code == 404:
                    # Asignamos valores predeterminados si la serie no se encuentra
                        where_to_watch = ["No disponible"]
                    elif donde_ver_response.status_code == 200:
                        donde_ver_info = donde_ver_response.json()["results"]
                        # sacamos las plataformas
                        for country, plataforma_info in donde_ver_info.items():
                            if "flatrate" in plataforma_info:
                                for plataforma in plataforma_info["flatrate"]:
                                        where_to_watch.append(plataforma["provider_name"])
                        # una vez añadidas todas las plataformas, elimininamos duplicados y quitamos la lista
                        where_to_watch = ", ".join(set(where_to_watch)) if where_to_watch else "No platforms available"
                    
                    # Solicitamos a la API las RESEÑAS de la serie
                    reviews_url = f"https://api.themoviedb.org/3/tv/{series_id}/reviews"
                    reviews_response = requests.get(reviews_url, params=params)
                    if reviews_response.status_code == 200:
                        reviews_data = reviews_response.json()
                        # Extraemos las reseñas
                        
                        for review in reviews_data["results"]:
                            review_content = review.get("content", "No review available")
                            reviews.append(review_content)
                    
                        # Si hay reseñas las unimos 
                        reviews_text = " | ".join(reviews) if reviews else "No reviews"
                    # reviews=[] 
                    # reviews=["muy bien", "perfecto"]
                    else:
                        reviews_text = "No reviews available"  # Si no hay reseñas, asignamos un texto predeterminado

                    # Solicitamos a la API los CREDITOS (actores principales y directores)
                    credits_url = f"https://api.themoviedb.org/3/tv/{series_id}/credits"
                    credits_response = requests.get(credits_url, params=params)
                    if credits_response.status_code == 200:
                        credits = credits_response.json()
                        # Obtener los primeros actores principales
                        cast = [actor["name"] for actor in credits.get("cast")[:5]]  # Primeros 5 actores
                    

                    # Solicitamos a la API los TRAILERS
                    videos_url = f"https://api.themoviedb.org/3/tv/{series_id}/videos"
                    videos_response = requests.get(videos_url, params=params)
                    if videos_response.status_code == 200:
                        videos = videos_response.json()["results"]
                        trailer_url = next((video for video in videos if video["type"] == "Trailer"), None)
                        if trailer_url:
                            trailer = f"https://www.youtube.com/watch?v={trailer_url['key']}"
                        else:
                            trailer = "No trailer available"

                    # Añadimos la serie al DataFrame
                    todas_series.append({
                        "Title": title,
                        "Synopsis": synopsis,
                        "Tagline": tagline,
                        "Genero": genero,
                        "Rating": rating,
                        "Popularity": popularity,
                        "Vote Count": number_ratings,
                        "Vote Average": vote_average,
                        "Original Language": original_language,
                        "Origin Country": origin_country,
                        "First Air Date": first_air_date,
                        "Last Air Date": last_air_date,
                        "In production": in_production,
                        "Status": status,
                        "Number of Season": number_of_seasons,
                        "Number of Episodes": number_of_episodes,
                        "Episode Duration": duracion_episodio,
                        "Production Companies": production_companies,
                        "Where to Watch": where_to_watch,
                        "Reviews": reviews_text,
                        "Budget": presupuesto,
                        "Revenue": recaudacion,
                        "Cast": ", ".join(cast),  # Unimos los nombres de los actores principales
                        "Creators": created_by,
                        "Poster": poster_path,
                        "Trailer": trailer
                    })
            else:
                print(f"Error {response.status_code}: No se pudo obtener los datos de la página {page}")
            print(f"Obteniendo datos de la página {page}... total de: {len(todas_series)} series")
        except requests.exceptions.RequestException as e:
            print(f"Error al obtener series de la página {page}: {e}")
            continue  # Continuar con la siguiente página si ha habido un error
    df = pd.DataFrame(todas_series)
    return df



In [ ]:
# Llamar a la función
df = get_all_series(500)  # Número de páginas que quiero obtener



Obteniendo datos de la página 1... total de: 20 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 2... total de: 40 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 3... total de: 55 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 4... total de: 74 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 5... total de: 90 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 6... total de: 104 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 7... total de: 121 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 8... total de: 137 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 9... total de: 155 series
Límite de peticiones alcanzado. Esperando...
Obteniendo datos de la página 10... total de: 170 series
Límite de peticiones alcanzado. Esperan

In [ ]:
df.to_csv("series9700.csv")